# Lesson 5: Multiple Tables and Relationships

**Duration:** 20 minutes  
**Prerequisites:** Complete Lessons 1-4  
**Learning Mode:** Read explanations, then run each SQL query

---

## 🎯 Learning Objectives

By the end of this lesson, you will be able to:
- Understand database normalization and why it matters
- Design one-to-many relationships
- Create related tables with foreign keys
- Link tables together properly
- Avoid data duplication
- Build many-to-many relationships with junction tables


## 📚 Why Use Multiple Tables?

Currently, our `characters` table stores homeworld as TEXT. This creates problems:

### Problems with Single Table Design:

1. **Duplication:** "Tatooine" is stored multiple times
2. **Inconsistency:** One entry might say "Tatooine", another "tatooine"
3. **Limited information:** We can't store climate, population, etc.
4. **Update issues:** Changing planet name requires updating many rows

### Solution: Use Multiple Related Tables!

Instead of storing planet names repeatedly, we:
- Create a `planets` table with all planet information
- Store only a planet ID in the `characters` table
- Link characters to planets using that ID


## 🏗️ Understanding Database Relationships

### One-to-Many Relationship

**Definition:** One record in Table A relates to many records in Table B.

**Examples:**
- **One planet → Many characters** (many characters from one planet)
- One customer → Many orders
- One author → Many books

### Many-to-Many Relationship

**Definition:** Multiple records in Table A relate to multiple records in Table B.

**Examples:**
- **Many characters → Many vehicles** (characters pilot multiple vehicles, vehicles have multiple pilots)
- Many students → Many courses
- Many actors → Many movies


## 🔑 Primary Key vs Foreign Key

| Key Type | Purpose | Example |
|----------|---------|---------|
| **Primary Key** | Uniquely identifies each row in a table | `id` in `planets` table |
| **Foreign Key** | References a primary key in another table | `homeworld_id` in `characters` table |

### Visual Example:

```md
characters table:
id | name           | homeworld_id (FOREIGN KEY)
1  | Luke Skywalker | 1
2  | Darth Vader    | 1

planets table:
id (PRIMARY KEY) | name     | climate
1                | Tatooine | arid
2                | Alderaan | temperate
```

Both Luke and Vader have `homeworld_id = 1`, which points to Tatooine.


## 🛠️ Setup: Connect to Database

**Run the next 2 cells:**

In [ ]:
# Load SQL magic extension
%load_ext sql

# Fix prettytable compatibility issue
import prettytable
try:
    # Try to access DEFAULT to see if it exists
    _ = prettytable.DEFAULT
except AttributeError:
    # If it doesn't exist, add it using SINGLE_BORDER
    from prettytable import SINGLE_BORDER
    prettytable.DEFAULT = SINGLE_BORDER

# Configure SQL magic settings
%config SqlMagic.autopandas = False
%config SqlMagic.displaycon = False
%config SqlMagic.feedback = False

In [ ]:
%sql sqlite:///starwars.db

## 🪐 Part 1: Create the Planets Table

### Create Planets Table with Schema

**New Concept:** `UNIQUE` constraint ensures no duplicate planet names.


In [ ]:
%%sql
-- Create planets table
CREATE TABLE IF NOT EXISTS planets (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL UNIQUE,
    climate TEXT,
    terrain TEXT,
    population INTEGER
);


### Insert Planet Data


In [ ]:
%%sql
-- Insert planets
INSERT INTO planets (name, climate, terrain, population) VALUES
    ('Tatooine', 'arid', 'desert', 200000),
    ('Alderaan', 'temperate', 'grasslands, mountains', 2000000000),
    ('Hoth', 'frozen', 'tundra, ice caves', NULL),
    ('Kashyyyk', 'tropical', 'jungle, forests', 45000000),
    ('Naboo', 'temperate', 'grassy hills, swamps', 4500000000),
    ('Corellia', 'temperate', 'plains, urban', 3000000000),
    ('Stewjon', 'temperate', 'grass', NULL),
    ('Unknown', NULL, NULL, NULL);


### View Planets Table


In [ ]:
%%sql
SELECT * FROM planets;


## 🔗 Part 2: Link Characters to Planets

### Add homeworld_id Column to Characters

**Note:** This creates the foreign key reference.


In [ ]:
%%sql
-- Add homeworld_id column
ALTER TABLE characters ADD COLUMN homeworld_id INTEGER;


### Update Characters with Planet IDs

**Explanation:** These subqueries find each planet's ID and update the character's `homeworld_id`.

**Run the next cell to link all characters to their planets:**


In [ ]:
%%sql
-- Update characters with homeworld_id
UPDATE characters SET homeworld_id = (SELECT id FROM planets WHERE name = 'Tatooine') WHERE homeworld = 'Tatooine';
UPDATE characters SET homeworld_id = (SELECT id FROM planets WHERE name = 'Alderaan') WHERE homeworld = 'Alderaan';
UPDATE characters SET homeworld_id = (SELECT id FROM planets WHERE name = 'Corellia') WHERE homeworld = 'Corellia';
UPDATE characters SET homeworld_id = (SELECT id FROM planets WHERE name = 'Kashyyyk') WHERE homeworld = 'Kashyyyk';
UPDATE characters SET homeworld_id = (SELECT id FROM planets WHERE name = 'Stewjon') WHERE homeworld = 'Stewjon';
UPDATE characters SET homeworld_id = (SELECT id FROM planets WHERE name = 'Naboo') WHERE homeworld = 'Naboo';
UPDATE characters SET homeworld_id = (SELECT id FROM planets WHERE name = 'Unknown') WHERE homeworld = 'Unknown';


### Verify the Links


In [1]:
%%sql
-- View updated characters with foreign keys
SELECT id, name, homeworld, homeworld_id FROM characters;


UsageError: Cell magic `%%sql` not found.


## 📊 Understanding Normalization

**Normalization** is the process of organizing data to reduce redundancy.

### Before (One Table):

```
id | name          | homeworld | climate | population
1  | Luke Skywalker| Tatooine  | arid    | 200000
2  | Darth Vader   | Tatooine  | arid    | 200000
```

❌ **Problem:** "Tatooine", "arid", "200000" stored twice (duplication!)

### After (Two Tables):

```
characters:
id | name          | homeworld_id
1  | Luke Skywalker| 1
2  | Darth Vader   | 1

planets:
id | name     | climate | population
1  | Tatooine | arid    | 200000
```

✅ **Benefit:** Planet data stored once, referenced multiple times!

### Benefits of Normalization:

1. **No duplication** - Store each fact once
2. **Consistency** - One source of truth
3. **Easier updates** - Change data in one place
4. **Data integrity** - Foreign keys enforce valid relationships


## 🚗 Part 3: Create Vehicles Table

Let's create a vehicles table for starships and speeders.


In [ ]:
%%sql
-- Create vehicles table
CREATE TABLE IF NOT EXISTS vehicles (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    model TEXT,
    vehicle_class TEXT,
    manufacturer TEXT
);


### Insert Vehicles


In [ ]:
%%sql
-- Insert vehicles
INSERT INTO vehicles (name, model, vehicle_class, manufacturer) VALUES
    ('X-wing', 'T-65 X-wing', 'Starfighter', 'Incom Corporation'),
    ('Millennium Falcon', 'YT-1300 light freighter', 'Light freighter', 'Corellian Engineering Corporation'),
    ('TIE Fighter', 'Twin Ion Engine Fighter', 'Starfighter', 'Sienar Fleet Systems'),
    ('Imperial Speeder Bike', '74-Z speeder bike', 'Speeder', 'Aratech Repulsor Company'),
    ('Snowspeeder', 'T-47 airspeeder', 'Airspeeder', 'Incom Corporation'),
    ('Lambda Shuttle', 'Lambda-class shuttle', 'Transport', 'Sienar Fleet Systems'),
    ('AT-AT', 'All Terrain Armoured Transport', 'Assault walker', 'Kuat Drive Yards'),
    ('Jedi Starfighter', 'Delta-7 Aethersprite', 'Starfighter', 'Kuat Systems Engineering');


### View Vehicles


In [ ]:
%%sql
SELECT * FROM vehicles;


## 🔀 Part 4: Many-to-Many Relationships

### The Problem:

- Luke flies multiple vehicles (X-wing, Snowspeeder)
- Multiple characters fly the Millennium Falcon (Han, Chewie)
- One character → Many vehicles
- One vehicle → Many characters
- This is a **many-to-many** relationship!

### The Solution: Junction Table

A **junction table** (also called linking table or associative table) connects two tables in a many-to-many relationship.

**Structure:**
```
character_vehicles:
character_id | vehicle_id
1            | 1          (Luke → X-wing)
1            | 5          (Luke → Snowspeeder)
3            | 2          (Han → Millennium Falcon)
4            | 2          (Chewie → Millennium Falcon)
```


### Create Junction Table


In [ ]:
%%sql
-- Create character_vehicles junction table
CREATE TABLE IF NOT EXISTS character_vehicles (
    character_id INTEGER NOT NULL,
    vehicle_id INTEGER NOT NULL,
    PRIMARY KEY (character_id, vehicle_id),
    FOREIGN KEY (character_id) REFERENCES characters(id),
    FOREIGN KEY (vehicle_id) REFERENCES vehicles(id)
);


**Explanation:**

- `character_id` references the `characters` table
- `vehicle_id` references the `vehicles` table
- `PRIMARY KEY (character_id, vehicle_id)` ensures each pairing is unique (can't link Luke to X-wing twice)
- Both columns are foreign keys to their respective tables


### Link Characters to Vehicles

**Important:** Make sure you use the correct IDs from your tables!


In [ ]:
%%sql
-- Link characters to vehicles (many-to-many relationship)
INSERT INTO character_vehicles (character_id, vehicle_id) VALUES
    -- Luke flies X-wing, Snowspeeder
    (1, 1),
    (1, 5),
    -- Han flies Millennium Falcon
    (3, 2),
    -- Chewbacca also flies Millennium Falcon
    (4, 2),
    -- Obi-Wan flies Jedi Starfighter
    (5, 8),
    -- Darth Vader flies TIE Fighter, Lambda Shuttle
    (6, 3),
    (6, 6),
    -- R2-D2 is IN X-wing and Jedi Starfighter
    (8, 1),
    (8, 8);


### View Character-Vehicle Links


In [ ]:
%%sql
SELECT * FROM character_vehicles;


## 🔍 Part 5: Verify Relationships

### Count Characters per Planet


In [ ]:
%%sql
-- How many characters from each planet?
SELECT p.name AS planet, COUNT(c.id) AS character_count
FROM planets p
LEFT JOIN characters c ON p.id = c.homeworld_id
GROUP BY p.name
ORDER BY character_count DESC;


**Note:** We're using a JOIN here (Lesson 6 topic), but it helps verify our relationships!

### Count Vehicles per Character


In [ ]:
%%sql
-- How many vehicles does each character pilot?
SELECT c.name AS character, COUNT(cv.vehicle_id) AS vehicle_count
FROM characters c
LEFT JOIN character_vehicles cv ON c.id = cv.character_id
GROUP BY c.name
ORDER BY vehicle_count DESC;


## 📋 Relationship Types Summary

### One-to-Many

**Setup:**
- Add foreign key column to the "many" table
- Points to primary key in the "one" table

**Example:** characters.homeworld_id → planets.id

### Many-to-Many

**Setup:**
- Create junction table
- Two foreign key columns (one for each table)
- Composite primary key on both columns

**Example:** character_vehicles (character_id, vehicle_id)


## 🎓 Practice Exercise

Add 2 more planets, 2 more vehicles, and create some new relationships!


### Exercise 1: Add More Planets


In [ ]:
%%sql
-- Practice: Add 2 more planets
INSERT INTO planets (name, climate, terrain, population) VALUES
    ('Endor', 'temperate', 'forests, mountains', 30000000),
    ('Bespin', 'temperate', 'gas giant', 6000000);


### Exercise 2: Add More Vehicles


In [ ]:
%%sql
-- Practice: Add 2 more vehicles
INSERT INTO vehicles (name, model, vehicle_class, manufacturer) VALUES
    ('Slave I', 'Firespray-31', 'Patrol craft', 'Kuat Systems Engineering'),
    ('Naboo Starfighter', 'N-1 starfighter', 'Starfighter', 'Theed Palace Space Vessel Engineering Corps');


### Exercise 3: Link New Vehicles to Characters


In [ ]:
%%sql
-- Practice: Link some characters to new vehicles
-- Example: Leia (id=2) flies Naboo Starfighter (id=10)
INSERT INTO character_vehicles (character_id, vehicle_id) VALUES
    (2, 10);


### Exercise 4: Verify Your Additions


In [ ]:
%%sql
-- Check new planets
SELECT * FROM planets WHERE name IN ('Endor', 'Bespin');


In [ ]:
%%sql
-- Check new vehicles
SELECT * FROM vehicles WHERE id >= 9;


## 🐛 Common Errors & Troubleshooting

### Error: "FOREIGN KEY constraint failed"

**Problem:** Trying to insert a character_id or vehicle_id that doesn't exist.

**Solution:** Verify the IDs exist:


In [ ]:
%%sql
-- Check valid character IDs
SELECT id, name FROM characters;


In [ ]:
%%sql
-- Check valid vehicle IDs
SELECT id, name FROM vehicles;


### Error: "UNIQUE constraint failed: planets.name"

**Problem:** Trying to insert a planet that already exists.

**Solution:** Check existing planets first:


In [ ]:
%%sql
SELECT name FROM planets;


### Wrong Foreign Key Values

**Problem:** `homeworld_id` doesn't match actual planet IDs.

**Solution:** Use subqueries to find correct IDs:


In [ ]:
%%sql
-- Find planet ID by name
SELECT id FROM planets WHERE name = 'Tatooine';


### Many-to-Many Confusion

**Remember:** 

**One-to-Many:**
- Use foreign key in the "many" table
- Example: characters.homeworld_id → planets.id

**Many-to-Many:**
- Use junction table with two foreign keys
- Example: character_vehicles (character_id, vehicle_id)

**Wrong Approach:**
```sql
-- ❌ DON'T DO THIS
ALTER TABLE characters ADD COLUMN vehicle_id INTEGER;
```

This only allows one vehicle per character!

**Correct Approach:**
```sql
-- ✅ DO THIS
CREATE TABLE character_vehicles (
    character_id INTEGER,
    vehicle_id INTEGER
);
```

This allows multiple vehicles per character and vice versa.


## 🎯 Challenge Problem (Optional)

**Task:** Design and create a `missions` table that tracks Star Wars missions. Each mission should have:
- A unique ID
- A name
- A location (foreign key to planets)
- A date
- A description

Then create a `character_missions` junction table to track which characters participated in which missions. Insert at least 3 missions and link characters to them.

**Requirements:**
- CREATE TABLE for missions
- CREATE TABLE for character_missions (junction table)
- INSERT 3+ missions
- INSERT 5+ character-mission links
- Verify with SELECT queries


In [ ]:
%%sql
-- Challenge Solution: Create missions table
CREATE TABLE IF NOT EXISTS missions (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    location_id INTEGER,
    mission_date TEXT,
    description TEXT,
    FOREIGN KEY (location_id) REFERENCES planets(id)
);


In [ ]:
%%sql
-- Challenge: Create character_missions junction table
CREATE TABLE IF NOT EXISTS character_missions (
    character_id INTEGER NOT NULL,
    mission_id INTEGER NOT NULL,
    PRIMARY KEY (character_id, mission_id),
    FOREIGN KEY (character_id) REFERENCES characters(id),
    FOREIGN KEY (mission_id) REFERENCES missions(id)
);


In [ ]:
%%sql
-- Challenge: Insert missions
INSERT INTO missions (name, location_id, mission_date, description) VALUES
    ('Battle of Yavin', (SELECT id FROM planets WHERE name = 'Unknown'), '0 BBY', 'Destroy the Death Star'),
    ('Battle of Hoth', (SELECT id FROM planets WHERE name = 'Hoth'), '3 ABY', 'Evacuate Echo Base'),
    ('Rescue on Naboo', (SELECT id FROM planets WHERE name = 'Naboo'), '32 BBY', 'Liberate Naboo from Trade Federation');


In [ ]:
%%sql
-- Challenge: Link characters to missions
INSERT INTO character_missions (character_id, mission_id) VALUES
    (1, 1),  -- Luke at Yavin
    (1, 2),  -- Luke at Hoth
    (2, 2),  -- Leia at Hoth
    (3, 1),  -- Han at Yavin
    (3, 2),  -- Han at Hoth
    (5, 3),  -- Obi-Wan at Naboo
    (9, 3);  -- Padmé at Naboo


In [ ]:
%%sql
-- Challenge: Verify missions and participants
SELECT 
    m.name AS mission,
    p.name AS location,
    c.name AS participant
FROM missions m
JOIN planets p ON m.location_id = p.id
JOIN character_missions cm ON m.id = cm.mission_id
JOIN characters c ON cm.character_id = c.id
ORDER BY m.name, c.name;


## ✅ Checkpoint & Summary

### What You've Learnt

- ✅ Understand database normalization (avoid duplication)
- ✅ Create related tables with foreign keys
- ✅ Design one-to-many relationships (planet → characters)
- ✅ Build many-to-many relationships (characters ↔ vehicles)
- ✅ Use junction tables for many-to-many
- ✅ Insert data maintaining referential integrity
- ✅ Link records across tables using IDs
- ✅ Verify relationships with queries

### Key SQL Commands

| Command | Purpose | Example |
|---------|---------|---------|
| `UNIQUE` | Prevent duplicates | `name TEXT NOT NULL UNIQUE` |
| `FOREIGN KEY` | Link to another table | `FOREIGN KEY (homeworld_id) REFERENCES planets(id)` |
| `PRIMARY KEY` | Unique identifier | `id INTEGER PRIMARY KEY AUTOINCREMENT` |
| Junction Table | Many-to-many link | `character_vehicles(character_id, vehicle_id)` |
| Subquery | Find ID | `(SELECT id FROM planets WHERE name = 'Tatooine')` |

### Key Concepts

| Concept | Meaning |
|---------|---------|
| **Normalization** | Organizing data to reduce duplication |
| **Primary Key** | Unique identifier for rows in a table |
| **Foreign Key** | Field linking to another table's primary key |
| **One-to-Many** | One record relates to many records |
| **Many-to-Many** | Multiple records relate to multiple records |
| **Junction Table** | Links two tables in many-to-many relationship |
| **Referential Integrity** | Ensuring foreign keys reference valid records |

## 🎉 Excellent Work!

You've now built a proper relational database with multiple connected tables! In the next lesson, you'll learn how to retrieve data from multiple tables using JOINs.

**Ready to continue?** Open `lesson6_joins.ipynb`

---

## 💾 Git Commands (for reference)

```bash
git status
git add solutions/lesson5_schema.ipynb
git commit -m "Completed Lesson 5: Multiple tables and relationships"
git push
```
